In [ ]:
!pip -q install langchain tiktoken pypdf sentence_transformers InstructorEmbedding faiss-cpu
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [ ]:
!pip install protobuf==3.20.3
# !pip install google-generativeai

In [ ]:
# Install specific package versions
# !pip install langchain
# !pip install python-dotenv==1.0.0
# !pip install streamlit==1.22.0
# !pip install tiktoken
# !pip install faiss-cpu
# !pip install protobuf
# !pip -q pypdf sentence_transformers InstructorEmbedding

In [ ]:
# !pip -q install langchain tiktoken pypdf sentence_transformers InstructorEmbedding
# !pip install -U sentence-transformers

In [ ]:
# !pip -q install pypdf

In [ ]:
# !pip show langchain

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
# Load and process the text files
loader = DirectoryLoader('/content/drive/MyDrive/AI-Chatbot/Documents/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load_and_split()
pages = loader.load()

In [ ]:
print(pages[0].page_content[0:500])
print(pages[0].metadata)

 
 
 
  
Document No: IT-004 
Page: 1 of 3 
Policy IT Acceptable Use 
 
   
This document is confidential and property of OVP and may not be duplicated, altered or removed from the facility. 
1. Purpose 
It is everyone’s responsibility to protect OVP Biotech’ intellectual, employee, customer, and partner information. 
The purpose of this policy is to define OVPs acceptable use of Information Technology to conduct business on 
company devices or network.  
 
2. Scope 
This policy applies to all e
{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 0}


In [ ]:
len(documents)

11

In [ ]:
documents[0]

Document(metadata={'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 0}, page_content='Document No: IT-004 \nPage: 1 of 3 \nPolicy IT Acceptable Use \n \n   \nThis document is confidential and property of OVP and may not be duplicated, altered or removed from the facility. \n1. Purpose \nIt is everyone’s responsibility to protect OVP Biotech’ intellectual, employee, customer, and partner information. \nThe purpose of this policy is to define OVPs acceptable use of Information Technology to conduct business on \ncompany devices or network.  \n \n2. Scope \nThis policy applies to all employees, contractors, and partners that have or require access to OVP systems or data. \n \n3. Responsibility \nRole Responsibilities  \nAll Employees \nResponsible for taking all necessary precautions to protect OVP Biotech’ intellec-\ntual, employee, customer, and partner information and equipment \n \n4. Definitions \nNone \n5. Procedure \n5.1. Use of IT Syste

In [ ]:

# from langchain.embeddings import HuggingFaceInstructEmbeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

instructor_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
e = instructor_embeddings.embed_query("What is the Purpose?")

In [ ]:
len(e)

384

In [ ]:
e[:5]

[-0.03824737295508385,
 0.10440891981124878,
 -0.026577139273285866,
 -0.014916379004716873,
 0.043447669595479965]

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=instructor_embeddings
)

# retriever = vectorstore.as_retriever(score_threshold = 0.7, search_type="similarity", search_kwargs={"k": 1})
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
res = vectorstore.similarity_search_with_score("What is the Role and Responsibilities of All Employees?", k=2)

for r in res:
    print(r[0].metadata)
    print(r[0].page_content)
    print("Score:", r[1])
    print()

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}
employee need to contravene these guidelines to perform their role, they should consult with, and ob-
tain approval from their manager before proceeding.  
5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening 
copyrights and patents, and using illegal or unlicensed software or services.   
5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information 
outside the company. 
5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of 
the business.
Score: 0.3954067081169252

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 0}
Document No: IT-004 
Page: 1 of 3 
Policy IT Acceptable Use 
 
   
This document is confidential and property of OVP and may not be duplicated, altered or removed from the faci

In [ ]:
res = vectorstore.similarity_search_with_score("Write the steps for Unacceptable Use?", k=2)

for r in res:
    print(r[0].metadata)
    print(r[0].page_content)
    print("Score:", r[1])
    print()

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}
employee need to contravene these guidelines to perform their role, they should consult with, and ob-
tain approval from their manager before proceeding.  
5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening 
copyrights and patents, and using illegal or unlicensed software or services.   
5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information 
outside the company. 
5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of 
the business.
Score: 0.48245590648700337

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}
nerable, users should take reasonable precautions to secure devices entrusted to their care. 
5.2.7. Users must guard against the risk of malware (e.g., viruses, spyware, Troj

In [ ]:
# from langchain.vectorstores import FAISS

# # Create a FAISS instance for vector database from 'data'
# vectordb = FAISS.from_documents(documents=documents,
#                                  embedding=instructor_embeddings)

# # Create a retriever for querying the vector database
# retriever = vectordb.as_retriever(score_threshold = 0.7, search_type="similarity", search_kwargs={"k": 1})

In [ ]:
rdocs = retriever.invoke("What is the Role and Responsibilities of All Employees?")
rdocs

[Document(id='c6fce13a-c2a7-49c0-bd65-2dc72cfd7ee1', metadata={'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}, page_content='employee need to contravene these guidelines to perform their role, they should consult with, and ob-\ntain approval from their manager before proceeding.  \n5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening \ncopyrights and patents, and using illegal or unlicensed software or services.   \n5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information \noutside the company. \n5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of \nthe business.')]

In [ ]:
rdocs[0].metadata

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf',
 'page': 1}

In [ ]:
print(rdocs[0].page_content)

employee need to contravene these guidelines to perform their role, they should consult with, and ob-
tain approval from their manager before proceeding.  
5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening 
copyrights and patents, and using illegal or unlicensed software or services.   
5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information 
outside the company. 
5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of 
the business.


In [ ]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
# 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

In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

··········


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What is the Role and Responsibilities of All Employees?"})

results

{'input': 'What is the Role and Responsibilities of All Employees?',
 'context': [Document(id='93148770-81b1-40b3-95fc-2cf08d9e3d86', metadata={'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}, page_content='employee need to contravene these guidelines to perform their role, they should consult with, and ob-\ntain approval from their manager before proceeding.  \n5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening \ncopyrights and patents, and using illegal or unlicensed software or services.   \n5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information \noutside the company. \n5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of \nthe business.'),
  Document(id='eab02311-1c2c-483b-bd9b-a3971a25f6e4', metadata={'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acce

In [ ]:
print(results["context"][0].page_content)

employee need to contravene these guidelines to perform their role, they should consult with, and ob-
tain approval from their manager before proceeding.  
5.3.1.1. All illegal activities. These include theft, computer hacking, malware distribution, contravening 
copyrights and patents, and using illegal or unlicensed software or services.   
5.3.1.2. All activities detrimental to the success of the company, including sharing sensitive information 
outside the company. 
5.3.1.3. All activities for personal benefit only that have a negative impact on the day-to-day functioning of 
the business.


In [ ]:
print(results["context"][0].metadata)

{'source': '/content/drive/MyDrive/AI-Chatbot/Documents/IT-004 IT Acceptable Use.pdf', 'page': 1}


In [ ]:
# !pip install -U google-generativeai
# !pip install --upgrade google-generativeai
# !pip install -U langchain

In [ ]:
import langchain
dir(langchain.llms)

['Any',
 'BaseLLM',
 'Callable',
 'Dict',
 'LangChainDeprecationWarning',
 'Type',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_import_ai21',
 '_import_aleph_alpha',
 '_import_amazon_api_gateway',
 '_import_anthropic',
 '_import_anyscale',
 '_import_arcee',
 '_import_aviary',
 '_import_azure_openai',
 '_import_azureml_endpoint',
 '_import_baidu_qianfan_endpoint',
 '_import_bananadev',
 '_import_baseten',
 '_import_beam',
 '_import_bedrock',
 '_import_bittensor',
 '_import_cerebriumai',
 '_import_chatglm',
 '_import_clarifai',
 '_import_cohere',
 '_import_ctransformers',
 '_import_ctranslate2',
 '_import_databricks',
 '_import_databricks_chat',
 '_import_deepinfra',
 '_import_deepsparse',
 '_import_edenai',
 '_import_fake',
 '_import_fireworks',
 '_import_forefrontai',
 '_import_gigachat',
 '_import_google_palm',
 '_import_gooseai',
 '_import_gpt4all',
 '_import_gradient_ai',